<h2>Imports & variable declarations </h2>

In [309]:
import glob
import cv2
import numpy as np
import os
from os.path import isfile, join

In [310]:
maximum_frames = 2443
minimum_frames = 113
video_path = "Clips/"

<h2>Functions</h2>

In [311]:
def read_videos(path):
    videos = []
    videos_fps= [] #frames per second
    videos_paths = glob.glob(path + '/*[0-9].mp4')
    for video_path in videos_paths:
        video = cv2.VideoCapture(video_path)
        fps = video.get(cv2.CAP_PROP_FPS)
        videos.append(video)
        videos_fps.append(fps)
    return videos, videos_fps

In [312]:
def read():
    path = 'Clips'
    truthful_videos, truth_videos_fps = read_videos(path + '/Truthful')
    deceptive_videos, deceptive_videos_fps = read_videos(path + '/Deceptive')
    return truthful_videos, truth_videos_fps, deceptive_videos, deceptive_videos_fps

In [313]:
def add_frames_with_noise(videos):
    videos_extended_noise = []
    for video in videos:
        frames_number = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
        if frames_number < maximum_frames:
            video = extend_video_with_noise(video)
        videos_extended_noise.append(video)
    return videos_extended_noise

In [323]:
def extend_video_with_noise(video):
    original_frames = get_video_frames(video)
    
    if len(original_frames) == 0:
        return video
    
    frames_with_noise = add_noise_to_frames(original_frames)
    
    final_frames = original_frames
    repeat_frames = maximum_frames // len(original_frames)
    i = 0
    while i in range(repeat_frames - 1):
        final_frames.extend(frames_with_noise)
        i += 1
    i = 0
    if len(final_frames) < maximum_frames:
        while len(final_frames) < maximum_frames and i < len(frames_with_noise):
            final_frames.append(frames_with_noise[i])
            i += 1
    return final_frames

In [315]:
def get_video_frames(video):
    frames = []
    success, frame = video.read()
    while success:
        frames.append(frame)
        success, frame = video.read()
    return frames

In [316]:
def add_noise_to_frames(frames):
    noisy_frames = []
    for frame in frames:
        noisy_frames.append(add_noise_to_frame(frame))
    return noisy_frames

In [317]:
def add_noise_to_frame(frame):
    mean = 0
    var = 20
    sigma = var ** 0.5
    #frame = resize_frame(frame)
    height, width, _ = frame.shape
    gaussian = np.random.normal(mean, sigma, (height, width)) #  np.zeros((224, 224), np.float32)
    noisy_image = np.zeros(frame.shape, np.float32)

    if len(frame.shape) == 2:
        noisy_image = img + gaussian
    else:
        noisy_image[:, :, 0] = frame[:, :, 0] + gaussian
        noisy_image[:, :, 1] = frame[:, :, 1] + gaussian
        noisy_image[:, :, 2] = frame[:, :, 2] + gaussian

    cv2.normalize(noisy_image, noisy_image, 0, 255, cv2.NORM_MINMAX, dtype=-1)
    noisy_frame = noisy_image.astype(np.uint8)
    
    #cv2.imshow("img", frame)
    #cv2.imshow("noise", noisy_frame)
    #cv2.waitKey(0)

    return noisy_frame

In [318]:
def convert_frames_to_videos(path, videos_frames, frames_per_second):
    videos = []
    for i, video_frames, fps in zip(range(1,len(videos_frames)+1), videos_frames, frames_per_second):
        video = create_video(path + "/{}.mp4".format(i), video_frames, fps)
        videos.append(video)
    return videos 

In [319]:
def create_video(pathOut, frames, fps):
    frame_array = []
    for i in range(len(frames)):
        height, width, layers = frames[i].shape
        size = (width,height)
        #inserting the frames into an image array
        frame_array.append(frames[i])

    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)

    for i in range(len(frame_array)):
        out.write(frame_array[i])
    out.release()

<h2>Main</h2>

In [320]:
truthful_videos, truth_videos_fps, deceptive_videos, deceptive_videos_fps = read()

In [324]:
truthful_videos_frames = add_frames_with_noise(truthful_videos)
deceptive_videos_frames = add_frames_with_noise(deceptive_videos)

In [322]:
truthful_videos_extended = convert_frames_to_videos(video_path + "Truthful-noise", truthful_videos_frames, truth_videos_fps)
deceptive_videos_extended = convert_frames_to_videos(video_path + "Deceptive-noise", deceptive_videos_frames, deceptive_videos_fps)